In [1]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, Input, Output
import plotly.express as px
from dash import dash_table
import base64
import pandas as pd
from Animal_Shelter import AnimalShelter  # Adjust according to your actual import

# Initialize the app
app = JupyterDash(__name__)

# Authentication
username = "aacuser"
password = "your_password_here"
db = AnimalShelter(username, password)
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

# Image processing
image_path = '/home/joshuadonnell_snhu/Desktop/customer-image.png'
encoded_image = base64.b64encode(open(image_path, 'rb').read()).decode()

app.layout = html.Div([
    html.H1('Grazioso Salvare Rescue Dashboard'),
    html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height': '100px'}),
    html.Hr(),
    dcc.RadioItems(
        id='rescue-type-filter',
        options=[
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain Rescue', 'value': 'Mountain'},
            {'label': 'Disaster Rescue', 'value': 'Disaster'},
            {'label': 'Reset Filters', 'value': 'reset'}
        ],
        value='reset',
        labelStyle={'display': 'inline-block'}
    ),
    html.Hr(),
    dash_table.DataTable(
        id='data-table',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Div(id='map-container'),
    html.Div([
        dcc.Graph(id='age-distribution-chart')
    ]),
    html.Hr(),
])

@app.callback(
    Output('data-table', 'data'),
    [Input('rescue-type-filter', 'value')]
)
def update_table(filter_value):
    if filter_value == 'reset':
        return df.to_dict('records')
    filtered_df = df[df['outcome_type'].str.contains(filter_value, na=False)]  # Check if 'outcome_type' matches your column
    return filtered_df.to_dict('records')

@app.callback(
    Output('map-container', "children"),
    [Input('data-table', "derived_virtual_data"),
     Input('data-table', "selected_rows")]
)
def update_map(rows, selected_rows):
    if not rows or not selected_rows:
        return []
    dff = pd.DataFrame(rows)
    index = selected_rows[0] if selected_rows else 0
    row = dff.iloc[index]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[row['location_lat'], row['location_long']], zoom=10, children=[
            dl.TileLayer(),
            dl.Marker(position=[row['location_lat'], row['location_long']], children=[
                dl.Tooltip(row['animal_type']),
                dl.Popup([html.H1("Animal Name"), html.P(row['name'])])
            ])
        ])
    ]

@app.callback(
    Output('age-distribution-chart', 'figure'),
    [Input('data-table', "selected_rows"),
     Input('data-table', "derived_virtual_data")]
)
def update_age_distribution(selected_rows, rows):
    if not rows or not selected_rows:
        return px.bar()  # Return an empty chart if no data is selected
    dff = pd.DataFrame(rows)
    # Use the first selected row if available, otherwise use the first row
    index = selected_rows[0] if selected_rows else 0
    row = dff.iloc[index]
    # Filter data to get similar types or use the whole dataset if none selected
    filtered_df = dff[dff['animal_type'] == row['animal_type']]
    fig = px.histogram(filtered_df, x='age_upon_outcome_in_weeks', nbins=20, title='Age Distribution')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)




Dash app running on http://127.0.0.1:13687/
